### Importing Libraries

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

### Load backtest results

In [2]:
from path_config import BACKTEST_RESULTS_DIR

trades = load_backtest_data(BACKTEST_RESULTS_DIR)

trades.head()

""


In [3]:
from pprint import pp

stats = load_backtest_stats(BACKTEST_RESULTS_DIR)

pp(stats.get('strategy'))

{'OrderFlowAnalysis': {'trades': [],
                       'locks': [],
                       'best_pair': {'key': 'BTC/USDT',
                                     'trades': 0,
                                     'profit_mean': 0.0,
                                     'profit_mean_pct': 0.0,
                                     'profit_sum': 0,
                                     'profit_sum_pct': 0.0,
                                     'profit_total_abs': 0,
                                     'profit_total': 0.0,
                                     'profit_total_pct': 0.0,
                                     'duration_avg': '0:00',
                                     'wins': 0,
                                     'draws': 0,
                                     'losses': 0,
                                     'winrate': 0.0},
                       'worst_pair': {'key': 'BTC/USDT',
                                      'trades': 0,
                                      '

### 1. Overall Performance

In [7]:
stats['strategy'][strategy].keys()

dict_keys(['trades', 'locks', 'best_pair', 'worst_pair', 'results_per_pair', 'results_per_enter_tag', 'exit_reason_summary', 'mix_tag_stats', 'left_open_trades', 'total_trades', 'trade_count_long', 'trade_count_short', 'total_volume', 'avg_stake_amount', 'profit_mean', 'profit_median', 'profit_total', 'profit_total_long', 'profit_total_short', 'profit_total_abs', 'profit_total_long_abs', 'profit_total_short_abs', 'cagr', 'expectancy', 'expectancy_ratio', 'sortino', 'sharpe', 'calmar', 'profit_factor', 'backtest_start', 'backtest_start_ts', 'backtest_end', 'backtest_end_ts', 'backtest_days', 'backtest_run_start_ts', 'backtest_run_end_ts', 'trades_per_day', 'market_change', 'pairlist', 'stake_amount', 'stake_currency', 'stake_currency_decimals', 'starting_balance', 'dry_run_wallet', 'final_balance', 'rejected_signals', 'timedout_entry_orders', 'timedout_exit_orders', 'canceled_trade_entries', 'canceled_entry_orders', 'replaced_entry_orders', 'max_open_trades', 'max_open_trades_setting', 

In [8]:
# Recalculate stats summary using the loaded stats
strategy = list(stats['strategy'].keys())[0]  # Assuming single strategy
summary_stats = stats['strategy'][strategy]

print("Summary of Backtest Results:")
print(f"Strategy Name: {summary_stats['strategy_name']}")
print(f"Timeframe: {summary_stats['timeframe']}")
print(f"Timerange: {summary_stats['timerange']}")
print(f"Starting Balance: {summary_stats['starting_balance']} {summary_stats['stake_currency']}")
print(f"Final Balance: {summary_stats['final_balance']:.2f} {summary_stats['stake_currency']}")
print(f"Total Profit: {summary_stats['profit_total_abs']:.2f} {summary_stats['stake_currency']} ({summary_stats['profit_total']:.2f}%)")
print(f"Total Trades: {summary_stats['total_trades']}")
print(f"Win Rate: {summary_stats['winrate']:.2f}%")
print(f"Profit Factor: {summary_stats['profit_factor']:.2f}")
print(f"Sharpe Ratio: {summary_stats['sharpe']:.2f}")
print(f"Sortino Ratio: {summary_stats['sortino']:.2f}")
print(f"CAGR: {summary_stats['cagr']:.2f}%")
print(f"Max Drawdown: {summary_stats['max_drawdown_account']:.2f}%")
print(f"Drawdown Start: {summary_stats['drawdown_start']}")
print(f"Drawdown End: {summary_stats['drawdown_end']}")
print(f"Market Change: {summary_stats['market_change']:.2f}%")
print(f"Trades per Day: {summary_stats['trades_per_day']:.2f}")
print(f"Avg. Stake Amount: {summary_stats['avg_stake_amount']:.2f} {summary_stats['stake_currency']}")
print(f"Avg. Trade Duration: {summary_stats['holding_avg']}")

Summary of Backtest Results:
Strategy Name: OrderFlowAnalysis
Timeframe: 1m
Timerange: 20230101-20240101
Starting Balance: 1000 USDT
Final Balance: 1000.00 USDT
Total Profit: 0.00 USDT (0.00%)
Total Trades: 0
Win Rate: 0.00%
Profit Factor: 0.00
Sharpe Ratio: 0.00
Sortino Ratio: 0.00
CAGR: 0.00%
Max Drawdown: 0.00%
Drawdown Start: 1970-01-01 00:00:00+00:00
Drawdown End: 1970-01-01 00:00:00+00:00
Market Change: 1.23%
Trades per Day: 0.00
Avg. Stake Amount: 0.00 USDT
Avg. Trade Duration: 0:00:00


### 2. Performance by pair

In [9]:
pair_performance = trades.groupby('pair').agg({
    'profit_ratio': ['mean', 'sum', 'count'],
    'profit_abs': 'sum'
}).reset_index()
pair_performance.columns = ['pair', 'avg_profit_ratio', 'total_profit_ratio', 'trade_count', 'total_profit_abs']
pair_performance = pair_performance.sort_values('total_profit_abs', ascending=False)

fig = px.bar(pair_performance, x='pair', y='total_profit_abs', 
             title='Total Profit by Pair',
             labels={'total_profit_abs': 'Total Profit', 'pair': 'Trading Pair'})
fig.show()

KeyError: 'pair'

### 3. Profit distribution

In [ ]:
fig = px.histogram(trades, x='profit_ratio', nbins=50,
                   title='Profit Ratio Distribution',
                   labels={'profit_ratio': 'Profit Ratio', 'count': 'Number of Trades'})
fig.show()

### 4. Cumulative returns over time

In [ ]:

trades_sorted = trades.sort_values('open_date')
trades_sorted['cumulative_profit'] = trades_sorted['profit_abs'].cumsum()

fig = px.line(trades_sorted, x='open_date', y='cumulative_profit',
              title='Cumulative Profit Over Time',
              labels={'cumulative_profit': 'Cumulative Profit', 'open_date': 'Date'})
fig.show()

### 5. Monthly returns

In [ ]:
trades['month'] = trades['open_date'].dt.to_period('M')
monthly_returns = trades.groupby('month')['profit_ratio'].sum().reset_index()
monthly_returns['month'] = monthly_returns['month'].astype(str)

fig = px.bar(monthly_returns, x='month', y='profit_ratio',
             title='Monthly Returns',
             labels={'profit_ratio': 'Profit Ratio', 'month': 'Month'})
fig.show()

### 6. Win rate over time

In [ ]:

trades_sorted['win'] = trades_sorted['profit_abs'] > 0
trades_sorted['cumulative_wins'] = trades_sorted['win'].cumsum()
trades_sorted['cumulative_trades'] = range(1, len(trades_sorted) + 1)
trades_sorted['win_rate'] = trades_sorted['cumulative_wins'] / trades_sorted['cumulative_trades']

fig = px.line(trades_sorted, x='open_date', y='win_rate',
              title='Win Rate Over Time',
              labels={'win_rate': 'Win Rate', 'open_date': 'Date'})
fig.show()

### 7. Average trade duration

In [ ]:

trades['duration'] = trades['close_date'] - trades['open_date']
avg_duration = trades['duration'].mean()
print(f"Average trade duration: {avg_duration}")

fig = px.histogram(trades, x='duration', nbins=50,
                   title='Trade Duration Distribution',
                   labels={'duration': 'Duration', 'count': 'Number of Trades'})
fig.show()

### 8. Correlation between profit and duration

In [ ]:
fig = px.scatter(trades, x='duration', y='profit_ratio',
                 title='Profit vs Duration',
                 labels={'duration': 'Duration', 'profit_ratio': 'Profit Ratio'})
fig.show()